In [2]:
#specify the packages of interest
options(warn=-1)
options(repr.plot.width=6, repr.plot.height=4)

packages = c("Matrix")

## Check to see if package is available and load else install the package and its dependencies
package.check <- lapply(packages, FUN = function(x) {
  if (!require(x, character.only = TRUE)) {
    install.packages(x, dependencies = TRUE)
    library(x, character.only = TRUE)
  }
})

Loading required package: Matrix


In [8]:
# Read in files (provide a full path, e.g. "~/Desktop/sample_dataset/users-likes.csv")

users <- read.csv("data/sample_dataset/users.csv")
likes <- read.csv("data/sample_dataset/likes.csv")
ul <- read.csv("data/sample_dataset/users-likes.csv") 

# Match entries in ul with users and likes dictionaries
ul$user_row<-match(ul$userid,users$userid)
ul$like_row<-match(ul$likeid,likes$likeid)

In [9]:
# Construct the sparse User-Like Matrix M
M <- sparseMatrix(i = ul$user_row, j = ul$like_row, x = 1)

# Save user IDs as row names in M
rownames(M) <- users$userid
    
# Save Like names as column names in M
colnames(M) <- likes$name
    
# Remove ul and likes objects (they won't be needed)
rm(ul, likes)

In [11]:
# Remove users/Likes occurring less than 150/80 times
 
repeat {                                       # repeat whatever is in the brackets
  i <- sum(dim(M))                             # check the size of M
  M <- M[rowSums(M) >= 150, colSums(M) >= 80]  # Retain only these rows/columns that meet the threshold
  if (sum(dim(M)) == i) break                  # if the size has not changed, break the loop
  }

# Check the new size of M
dim(M)

# Remove the users from users object that were removed
# from M
users <- users[match(rownames(M), users$userid), ]

# Check the new size of users
dim(users)

[1]  7258 11017

[1] 7258    9

In [15]:
set.seed(1122)

n <- nrow(users)
shuffled_df <- users[sample(n), ]
train_indices <- 1:round(0.6 * n)
train <- shuffled_df[train_indices, ]
test_indices <- (round(0.6 * n) + 1):n
test <- shuffled_df[test_indices, ]

In [14]:
names(users)

[1] "userid"    "gender"    "age"       "political" "ope"       "con"      
[7] "ext"       "agr"       "neu"